# Approach 4: Ensemble Methods

## Spotify Song Popularity Prediction

In the previous approaches, we found that **ElasticNet** achieved the best cross-validation RMSE (10.42). However, when we submitted predictions to Kaggle, the score didn't improve as expected.

**Why?** Different models capture different patterns in data:
- **Linear models** (Ridge, Lasso, ElasticNet) assume linear relationships
- **Tree-based models** (Random Forest, Gradient Boosting) capture non-linear patterns

**Solution:** Combine multiple models using ensemble methods to leverage the strengths of each.

---

### What We'll Cover:
1. Understanding ensemble methods (Voting, Stacking, Blending)
2. Comparing single models vs ensembles
3. Finding the best combination for Kaggle submission

---

### Previous Results:
| Approach | Best Model | CV RMSE |
|----------|------------|--------|
| Baseline | - | 11.27 |
| Approach 1 (top 15 genres) | Random Forest | 10.93 |
| Approach 2 (hybrid groups) | Ridge | 11.05 |
| Approach 3 v2 (leakage-safe) | ElasticNet | 10.42 |

---
## 1. Import Libraries

In [ ]:
# Core libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler

# Models
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor

# Metrics
from sklearn.metrics import mean_squared_error, r2_score

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries imported successfully!")

---
## 2. Understanding Ensemble Methods

Before we dive into the code, let's understand the three main ensemble approaches we'll use:

In [ ]:
# Visualize ensemble methods concept
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 1. Voting
ax1 = axes[0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.add_patch(plt.Rectangle((1, 6), 2, 2, color='steelblue', alpha=0.7))
ax1.add_patch(plt.Rectangle((4, 6), 2, 2, color='coral', alpha=0.7))
ax1.add_patch(plt.Rectangle((7, 6), 2, 2, color='green', alpha=0.7))
ax1.text(2, 7, 'Model 1', ha='center', va='center', fontsize=10, color='white', fontweight='bold')
ax1.text(5, 7, 'Model 2', ha='center', va='center', fontsize=10, color='white', fontweight='bold')
ax1.text(8, 7, 'Model 3', ha='center', va='center', fontsize=10, color='white', fontweight='bold')
ax1.annotate('', xy=(5, 4.5), xytext=(2, 5.8), arrowprops=dict(arrowstyle='->', color='black'))
ax1.annotate('', xy=(5, 4.5), xytext=(5, 5.8), arrowprops=dict(arrowstyle='->', color='black'))
ax1.annotate('', xy=(5, 4.5), xytext=(8, 5.8), arrowprops=dict(arrowstyle='->', color='black'))
ax1.add_patch(plt.Rectangle((3.5, 2.5), 3, 2, color='purple', alpha=0.7))
ax1.text(5, 3.5, 'AVERAGE', ha='center', va='center', fontsize=10, color='white', fontweight='bold')
ax1.annotate('', xy=(5, 1), xytext=(5, 2.3), arrowprops=dict(arrowstyle='->', color='black'))
ax1.add_patch(plt.Rectangle((3.5, 0), 3, 1, color='gold', alpha=0.9))
ax1.text(5, 0.5, 'Final Prediction', ha='center', va='center', fontsize=9, fontweight='bold')
ax1.set_title('VOTING\n(Simple Average)', fontsize=12, fontweight='bold')
ax1.axis('off')

# 2. Stacking
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.add_patch(plt.Rectangle((1, 6), 2, 2, color='steelblue', alpha=0.7))
ax2.add_patch(plt.Rectangle((4, 6), 2, 2, color='coral', alpha=0.7))
ax2.add_patch(plt.Rectangle((7, 6), 2, 2, color='green', alpha=0.7))
ax2.text(2, 7, 'Model 1', ha='center', va='center', fontsize=10, color='white', fontweight='bold')
ax2.text(5, 7, 'Model 2', ha='center', va='center', fontsize=10, color='white', fontweight='bold')
ax2.text(8, 7, 'Model 3', ha='center', va='center', fontsize=10, color='white', fontweight='bold')
ax2.annotate('', xy=(5, 4.5), xytext=(2, 5.8), arrowprops=dict(arrowstyle='->', color='black'))
ax2.annotate('', xy=(5, 4.5), xytext=(5, 5.8), arrowprops=dict(arrowstyle='->', color='black'))
ax2.annotate('', xy=(5, 4.5), xytext=(8, 5.8), arrowprops=dict(arrowstyle='->', color='black'))
ax2.add_patch(plt.Rectangle((3, 2.5), 4, 2, color='darkred', alpha=0.7))
ax2.text(5, 3.5, 'META-LEARNER', ha='center', va='center', fontsize=9, color='white', fontweight='bold')
ax2.annotate('', xy=(5, 1), xytext=(5, 2.3), arrowprops=dict(arrowstyle='->', color='black'))
ax2.add_patch(plt.Rectangle((3.5, 0), 3, 1, color='gold', alpha=0.9))
ax2.text(5, 0.5, 'Final Prediction', ha='center', va='center', fontsize=9, fontweight='bold')
ax2.set_title('STACKING\n(Meta-Learner Combines)', fontsize=12, fontweight='bold')
ax2.axis('off')

# 3. Blending
ax3 = axes[2]
ax3.set_xlim(0, 10)
ax3.set_ylim(0, 10)
ax3.add_patch(plt.Rectangle((1, 6), 2, 2, color='steelblue', alpha=0.7))
ax3.add_patch(plt.Rectangle((4, 6), 2, 2, color='coral', alpha=0.7))
ax3.add_patch(plt.Rectangle((7, 6), 2, 2, color='green', alpha=0.7))
ax3.text(2, 7, 'Model 1', ha='center', va='center', fontsize=10, color='white', fontweight='bold')
ax3.text(5, 7, 'Model 2', ha='center', va='center', fontsize=10, color='white', fontweight='bold')
ax3.text(8, 7, 'Model 3', ha='center', va='center', fontsize=10, color='white', fontweight='bold')
ax3.text(2, 5.5, '30%', ha='center', fontsize=9, color='steelblue', fontweight='bold')
ax3.text(5, 5.5, '50%', ha='center', fontsize=9, color='coral', fontweight='bold')
ax3.text(8, 5.5, '20%', ha='center', fontsize=9, color='green', fontweight='bold')
ax3.annotate('', xy=(5, 4.5), xytext=(2, 5.2), arrowprops=dict(arrowstyle='->', color='black'))
ax3.annotate('', xy=(5, 4.5), xytext=(5, 5.2), arrowprops=dict(arrowstyle='->', color='black'))
ax3.annotate('', xy=(5, 4.5), xytext=(8, 5.2), arrowprops=dict(arrowstyle='->', color='black'))
ax3.add_patch(plt.Rectangle((2.5, 2.5), 5, 2, color='darkorange', alpha=0.7))
ax3.text(5, 3.5, 'WEIGHTED AVG', ha='center', va='center', fontsize=9, color='white', fontweight='bold')
ax3.annotate('', xy=(5, 1), xytext=(5, 2.3), arrowprops=dict(arrowstyle='->', color='black'))
ax3.add_patch(plt.Rectangle((3.5, 0), 3, 1, color='gold', alpha=0.9))
ax3.text(5, 0.5, 'Final Prediction', ha='center', va='center', fontsize=9, fontweight='bold')
ax3.set_title('BLENDING\n(Weighted Average)', fontsize=12, fontweight='bold')
ax3.axis('off')

plt.suptitle('Three Ensemble Approaches', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Ensemble Methods Explained:

| Method | How It Works | Pros | Cons |
|--------|--------------|------|------|
| **Voting** | Averages predictions from all models equally | Simple, reduces variance | Treats all models equally |
| **Stacking** | Uses another model to learn how to combine predictions | Can learn optimal combination | Risk of overfitting |
| **Blending** | Weighted average with custom weights | Control over each model's contribution | Need to tune weights |

---
## 3. Load and Prepare Data

We'll use the same leakage-safe methodology from Approach 3 v2.

In [ ]:
# Load data
train_df = pd.read_csv('./data/CS98XRegressionTrain.csv')
test_df = pd.read_csv('./data/CS98XRegressionTest.csv')

# Handle missing genres
train_df['top genre'] = train_df['top genre'].fillna('Unknown').replace('', 'Unknown')
test_df['top genre'] = test_df['top genre'].fillna('Unknown').replace('', 'Unknown')

print("="*60)
print("DATA LOADED")
print("="*60)
print(f"Training set: {train_df.shape[0]} rows, {train_df.shape[1]} columns")
print(f"Test set:     {test_df.shape[0]} rows, {test_df.shape[1]} columns")

In [ ]:
# Constants and feature definitions
TOP_N_GENRES = 15
numerical_features = ['bpm', 'nrgy', 'dnce', 'dB', 'live', 'val', 'dur', 'acous', 'spch']

# Learn top genres from training data only
top_genres = train_df['top genre'].value_counts().head(TOP_N_GENRES).index.tolist()

print(f"Top {TOP_N_GENRES} genres (learned from training data):")
for i, genre in enumerate(top_genres, 1):
    print(f"  {i:2}. {genre}")

In [ ]:
def encode_genres(df, top_genres):
    """One-hot encode genres, grouping rare genres as 'other'."""
    df = df.copy()
    df['genre_simplified'] = df['top genre'].apply(
        lambda x: x if x in top_genres else 'other'
    )
    genre_dummies = pd.get_dummies(df['genre_simplified'], prefix='genre')
    df = pd.concat([df, genre_dummies], axis=1)
    return df

def engineer_features(df):
    """Create engineered features from numerical columns."""
    df = df.copy()
    
    # Interaction terms
    df['nrgy_x_dnce'] = df['nrgy'] * df['dnce']
    df['nrgy_x_val'] = df['nrgy'] * df['val']
    df['nrgy_x_dB'] = df['nrgy'] * df['dB']
    df['dnce_x_val'] = df['dnce'] * df['val']
    df['dnce_x_bpm'] = df['dnce'] * df['bpm']
    df['acous_x_nrgy'] = df['acous'] * df['nrgy']
    
    # Ratios
    df['nrgy_per_bpm'] = df['nrgy'] / (df['bpm'] + 1)
    df['dnce_per_nrgy'] = df['dnce'] / (df['nrgy'] + 1)
    df['val_per_nrgy'] = df['val'] / (df['nrgy'] + 1)
    df['spch_per_dur'] = df['spch'] / (df['dur'] + 1)
    
    # Polynomial features
    df['dur_squared'] = df['dur'] ** 2
    df['acous_squared'] = df['acous'] ** 2
    df['dB_squared'] = df['dB'] ** 2
    df['nrgy_squared'] = df['nrgy'] ** 2
    
    # Binned features
    df['bpm_slow'] = (df['bpm'] < 100).astype(int)
    df['bpm_medium'] = ((df['bpm'] >= 100) & (df['bpm'] < 130)).astype(int)
    df['bpm_fast'] = (df['bpm'] >= 130).astype(int)
    df['low_energy'] = (df['nrgy'] < 50).astype(int)
    df['high_energy'] = (df['nrgy'] >= 70).astype(int)
    df['is_acoustic'] = (df['acous'] > 50).astype(int)
    df['short_song'] = (df['dur'] < 180).astype(int)
    df['long_song'] = (df['dur'] > 300).astype(int)
    
    # Composite scores
    df['party_score'] = (df['nrgy'] + df['dnce'] + df['val'] - df['acous']) / 4
    df['chill_score'] = (df['acous'] + (100 - df['nrgy']) + (100 - df['dnce'])) / 3
    df['vocal_score'] = df['spch'] + df['live']
    
    return df

# Engineered features list
engineered_features = [
    'nrgy_x_dnce', 'nrgy_x_val', 'nrgy_x_dB', 'dnce_x_val', 'dnce_x_bpm', 'acous_x_nrgy',
    'nrgy_per_bpm', 'dnce_per_nrgy', 'val_per_nrgy', 'spch_per_dur',
    'dur_squared', 'acous_squared', 'dB_squared', 'nrgy_squared',
    'bpm_slow', 'bpm_medium', 'bpm_fast', 'low_energy', 'high_energy',
    'is_acoustic', 'short_song', 'long_song',
    'party_score', 'chill_score', 'vocal_score'
]

print("Feature engineering functions defined")
print(f"  - {len(numerical_features)} numerical features")
print(f"  - {len(engineered_features)} engineered features")

In [ ]:
# Apply encoding and feature engineering
train_enc = encode_genres(train_df, top_genres)
test_enc = encode_genres(test_df, top_genres)

train_fe = engineer_features(train_enc)
test_fe = engineer_features(test_enc)

# Get genre columns
genre_cols = [c for c in train_fe.columns if c.startswith('genre_') and c != 'genre_simplified']

# Align test columns with training
for col in genre_cols:
    if col not in test_fe.columns:
        test_fe[col] = 0

# Define complete feature set
features = numerical_features + genre_cols + engineered_features

# Prepare X and y
X = train_fe[features].copy()
y = train_fe['pop'].copy()
X_test = test_fe[features].copy()

# Ensure test has all features
for col in features:
    if col not in X_test.columns:
        X_test[col] = 0
X_test = X_test[features]

print("="*60)
print("FEATURES PREPARED")
print("="*60)
print(f"Total features: {len(features)}")
print(f"  - Numerical:   {len(numerical_features)}")
print(f"  - Genre:       {len(genre_cols)}")
print(f"  - Engineered:  {len(engineered_features)}")

In [ ]:
# Visualize feature breakdown
fig, ax = plt.subplots(figsize=(8, 6))

feature_counts = {
    'Numerical\n(original)': len(numerical_features),
    'Genre\n(one-hot)': len(genre_cols),
    'Engineered\n(interactions, ratios, etc.)': len(engineered_features)
}

colors = ['steelblue', 'coral', 'green']
bars = ax.bar(feature_counts.keys(), feature_counts.values(), color=colors, edgecolor='black')

# Add value labels
for bar, val in zip(bars, feature_counts.values()):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
            str(val), ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('Number of Features', fontsize=12)
ax.set_title(f'Feature Breakdown (Total: {len(features)} features)', fontsize=14, fontweight='bold')
ax.set_ylim(0, max(feature_counts.values()) + 5)

plt.tight_layout()
plt.show()

In [ ]:
# Scale features for linear models
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

print("Features scaled (for linear models)")
print("Note: Tree-based models will use unscaled features")

---
## 4. Single Model Performance (Baseline)

First, let's see how each individual model performs. This gives us a baseline to compare our ensembles against.

In [ ]:
# Define base models
base_models = {
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.1),
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
}

# Cross-validate each model
print("="*60)
print("SINGLE MODEL PERFORMANCE (5-fold CV)")
print("="*60)

base_results = []
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for name, model in base_models.items():
    cv_rmses = []
    cv_r2s = []
    
    for train_idx, val_idx in kf.split(X):
        # Use scaled data for linear models, raw for tree-based
        if name in ['Ridge', 'Lasso', 'ElasticNet']:
            X_tr, X_va = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
        else:
            X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
        
        y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_tr.values, y_tr.values)
        y_pred = model.predict(X_va.values)
        
        rmse = np.sqrt(mean_squared_error(y_va, y_pred))
        r2 = r2_score(y_va, y_pred)
        cv_rmses.append(rmse)
        cv_r2s.append(r2)
    
    cv_rmse = np.mean(cv_rmses)
    cv_std = np.std(cv_rmses)
    cv_r2 = np.mean(cv_r2s)
    
    base_results.append({
        'Model': name, 
        'CV RMSE': cv_rmse, 
        'Std': cv_std,
        'CV R2': cv_r2,
        'Type': 'Linear' if name in ['Ridge', 'Lasso', 'ElasticNet'] else 'Tree-based'
    })
    print(f"{name:20} CV RMSE: {cv_rmse:.4f} (+/- {cv_std:.4f})  R2: {cv_r2:.3f}")

base_df = pd.DataFrame(base_results).sort_values('CV RMSE')
print(f"\nBest single model: {base_df.iloc[0]['Model']} (RMSE: {base_df.iloc[0]['CV RMSE']:.4f})")

In [ ]:
# Visualize single model performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: CV RMSE comparison
ax1 = axes[0]
colors = ['steelblue' if t == 'Linear' else 'coral' for t in base_df['Type']]
bars = ax1.barh(base_df['Model'], base_df['CV RMSE'], xerr=base_df['Std'], 
                color=colors, edgecolor='black', capsize=5)
ax1.set_xlabel('CV RMSE (lower is better)', fontsize=11)
ax1.set_title('Single Model CV RMSE Comparison', fontsize=12, fontweight='bold')
ax1.invert_yaxis()

# Add value labels
for i, (bar, val) in enumerate(zip(bars, base_df['CV RMSE'])):
    ax1.text(val + 0.15, i, f'{val:.2f}', va='center', fontsize=10)

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='steelblue', label='Linear Models'),
                   Patch(facecolor='coral', label='Tree-based Models')]
ax1.legend(handles=legend_elements, loc='lower right')

# Plot 2: Linear vs Tree-based comparison
ax2 = axes[1]
linear_avg = base_df[base_df['Type'] == 'Linear']['CV RMSE'].mean()
tree_avg = base_df[base_df['Type'] == 'Tree-based']['CV RMSE'].mean()

bars2 = ax2.bar(['Linear Models\n(Ridge, Lasso, ElasticNet)', 'Tree-based Models\n(RF, Gradient Boosting)'],
                [linear_avg, tree_avg], color=['steelblue', 'coral'], edgecolor='black')
ax2.set_ylabel('Average CV RMSE', fontsize=11)
ax2.set_title('Linear vs Tree-based Models', fontsize=12, fontweight='bold')

for bar, val in zip(bars2, [linear_avg, tree_avg]):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{val:.2f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nLinear models average RMSE: {linear_avg:.4f}")
print(f"Tree-based models average RMSE: {tree_avg:.4f}")
print(f"Linear models are {tree_avg - linear_avg:.4f} RMSE better on average")

### Key Observation:
Linear models (especially ElasticNet) outperform tree-based models on CV. But CV isn't everything - let's see if combining them helps on the actual test set.

---
## 5. Voting Ensembles

Voting simply averages predictions from multiple models. It's the simplest ensemble method.

In [ ]:
# Define voting ensembles
voting_ensembles = {
    'Voting: Linear Only': VotingRegressor([
        ('ridge', Ridge(alpha=1.0)),
        ('lasso', Lasso(alpha=0.1)),
        ('elasticnet', ElasticNet(alpha=0.1, l1_ratio=0.5))
    ]),
    
    'Voting: Tree Only': VotingRegressor([
        ('rf', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)),
        ('gb', GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42))
    ]),
    
    'Voting: Mixed (Ridge+RF+GB)': VotingRegressor([
        ('ridge', Ridge(alpha=1.0)),
        ('rf', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)),
        ('gb', GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42))
    ]),
    
    'Voting: All 5 Models': VotingRegressor([
        ('ridge', Ridge(alpha=1.0)),
        ('lasso', Lasso(alpha=0.1)),
        ('elasticnet', ElasticNet(alpha=0.1, l1_ratio=0.5)),
        ('rf', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)),
        ('gb', GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42))
    ])
}

print("="*60)
print("VOTING ENSEMBLE PERFORMANCE")
print("="*60)

voting_results = []

for name, model in voting_ensembles.items():
    cv_rmses = []
    
    for train_idx, val_idx in kf.split(X_scaled):
        X_tr, X_va = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
        y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_tr.values, y_tr.values)
        y_pred = model.predict(X_va.values)
        rmse = np.sqrt(mean_squared_error(y_va, y_pred))
        cv_rmses.append(rmse)
    
    cv_rmse = np.mean(cv_rmses)
    cv_std = np.std(cv_rmses)
    voting_results.append({'Ensemble': name, 'CV RMSE': cv_rmse, 'Std': cv_std, 'Type': 'Voting'})
    print(f"{name:30} CV RMSE: {cv_rmse:.4f} (+/- {cv_std:.4f})")

voting_df = pd.DataFrame(voting_results).sort_values('CV RMSE')

In [ ]:
# Visualize voting results
fig, ax = plt.subplots(figsize=(10, 5))

y_pos = np.arange(len(voting_df))
bars = ax.barh(y_pos, voting_df['CV RMSE'], xerr=voting_df['Std'], 
               color='purple', alpha=0.7, edgecolor='black', capsize=5)

# Add best single model line
best_single = base_df.iloc[0]['CV RMSE']
ax.axvline(x=best_single, color='red', linestyle='--', linewidth=2, 
           label=f'Best Single Model ({best_single:.2f})')

ax.set_yticks(y_pos)
ax.set_yticklabels(voting_df['Ensemble'])
ax.set_xlabel('CV RMSE (lower is better)', fontsize=11)
ax.set_title('Voting Ensemble Performance', fontsize=12, fontweight='bold')
ax.invert_yaxis()
ax.legend(loc='lower right')

# Add value labels
for i, val in enumerate(voting_df['CV RMSE']):
    ax.text(val + 0.1, i, f'{val:.2f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## 6. Stacking Ensembles

Stacking uses a "meta-learner" to combine predictions from base models. The meta-learner learns the optimal way to combine predictions.

In [ ]:
# Define stacking ensembles
stacking_ensembles = {
    'Stacking: RF+GB -> Ridge': StackingRegressor(
        estimators=[
            ('rf', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)),
            ('gb', GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42))
        ],
        final_estimator=Ridge(alpha=1.0),
        cv=5
    ),
    
    'Stacking: Linear -> RF': StackingRegressor(
        estimators=[
            ('ridge', Ridge(alpha=1.0)),
            ('lasso', Lasso(alpha=0.1)),
            ('elasticnet', ElasticNet(alpha=0.1, l1_ratio=0.5))
        ],
        final_estimator=RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42),
        cv=5
    ),
    
    'Stacking: All -> Ridge': StackingRegressor(
        estimators=[
            ('ridge', Ridge(alpha=1.0)),
            ('elasticnet', ElasticNet(alpha=0.1, l1_ratio=0.5)),
            ('rf', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)),
            ('gb', GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42))
        ],
        final_estimator=Ridge(alpha=1.0),
        cv=5
    )
}

print("="*60)
print("STACKING ENSEMBLE PERFORMANCE")
print("="*60)

stacking_results = []

for name, model in stacking_ensembles.items():
    cv_rmses = []
    
    for train_idx, val_idx in kf.split(X_scaled):
        X_tr, X_va = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
        y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_tr.values, y_tr.values)
        y_pred = model.predict(X_va.values)
        rmse = np.sqrt(mean_squared_error(y_va, y_pred))
        cv_rmses.append(rmse)
    
    cv_rmse = np.mean(cv_rmses)
    cv_std = np.std(cv_rmses)
    stacking_results.append({'Ensemble': name, 'CV RMSE': cv_rmse, 'Std': cv_std, 'Type': 'Stacking'})
    print(f"{name:30} CV RMSE: {cv_rmse:.4f} (+/- {cv_std:.4f})")

stacking_df = pd.DataFrame(stacking_results).sort_values('CV RMSE')

---
## 7. Blending (Weighted Average)

Blending lets us assign custom weights to each model's predictions. This gives us fine-grained control.

In [ ]:
# Define blend configurations to test
blends = {
    'Equal Weight (All 5)': {
        'Ridge': 0.2, 'Lasso': 0.2, 'ElasticNet': 0.2, 
        'RandomForest': 0.2, 'GradientBoosting': 0.2
    },
    'Linear Only (Equal)': {
        'Ridge': 0.33, 'Lasso': 0.33, 'ElasticNet': 0.34, 
        'RandomForest': 0, 'GradientBoosting': 0
    },
    'Heavy ElasticNet': {
        'Ridge': 0.15, 'Lasso': 0.15, 'ElasticNet': 0.5, 
        'RandomForest': 0.1, 'GradientBoosting': 0.1
    },
    'ElasticNet + RF (60/40)': {
        'Ridge': 0, 'Lasso': 0, 'ElasticNet': 0.6, 
        'RandomForest': 0.4, 'GradientBoosting': 0
    },
    'ElasticNet + RF (70/30)': {
        'Ridge': 0, 'Lasso': 0, 'ElasticNet': 0.7, 
        'RandomForest': 0.3, 'GradientBoosting': 0
    },
    'Ridge + ElasticNet': {
        'Ridge': 0.5, 'Lasso': 0, 'ElasticNet': 0.5, 
        'RandomForest': 0, 'GradientBoosting': 0
    }
}

print("="*60)
print("BLENDING PERFORMANCE")
print("="*60)

blend_results = []

for blend_name, weights in blends.items():
    cv_rmses = []
    
    for train_idx, val_idx in kf.split(X):
        # Prepare fold data
        X_tr_scaled = X_scaled.iloc[train_idx]
        X_va_scaled = X_scaled.iloc[val_idx]
        X_tr_raw = X.iloc[train_idx]
        X_va_raw = X.iloc[val_idx]
        y_tr = y.iloc[train_idx]
        y_va = y.iloc[val_idx]
        
        # Train models and get predictions
        preds = {}
        
        if weights['Ridge'] > 0:
            m = Ridge(alpha=1.0)
            m.fit(X_tr_scaled.values, y_tr.values)
            preds['Ridge'] = m.predict(X_va_scaled.values)
        
        if weights['Lasso'] > 0:
            m = Lasso(alpha=0.1)
            m.fit(X_tr_scaled.values, y_tr.values)
            preds['Lasso'] = m.predict(X_va_scaled.values)
        
        if weights['ElasticNet'] > 0:
            m = ElasticNet(alpha=0.1, l1_ratio=0.5)
            m.fit(X_tr_scaled.values, y_tr.values)
            preds['ElasticNet'] = m.predict(X_va_scaled.values)
        
        if weights['RandomForest'] > 0:
            m = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
            m.fit(X_tr_raw.values, y_tr.values)
            preds['RandomForest'] = m.predict(X_va_raw.values)
        
        if weights['GradientBoosting'] > 0:
            m = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
            m.fit(X_tr_raw.values, y_tr.values)
            preds['GradientBoosting'] = m.predict(X_va_raw.values)
        
        # Blend predictions
        blend_pred = sum(weights[name] * preds[name] for name in preds)
        rmse = np.sqrt(mean_squared_error(y_va, blend_pred))
        cv_rmses.append(rmse)
    
    cv_rmse = np.mean(cv_rmses)
    cv_std = np.std(cv_rmses)
    blend_results.append({'Ensemble': blend_name, 'CV RMSE': cv_rmse, 'Std': cv_std, 'Type': 'Blending'})
    print(f"{blend_name:30} CV RMSE: {cv_rmse:.4f} (+/- {cv_std:.4f})")

blend_df = pd.DataFrame(blend_results).sort_values('CV RMSE')

In [ ]:
# Visualize blend weights for top blends
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

colors_map = {
    'Ridge': 'steelblue',
    'Lasso': 'lightblue', 
    'ElasticNet': 'navy',
    'RandomForest': 'coral',
    'GradientBoosting': 'lightsalmon'
}

for i, (blend_name, weights) in enumerate(blends.items()):
    ax = axes[i]
    active_models = {k: v for k, v in weights.items() if v > 0}
    
    colors = [colors_map[m] for m in active_models.keys()]
    wedges, texts, autotexts = ax.pie(
        active_models.values(), 
        labels=active_models.keys(),
        autopct='%1.0f%%',
        colors=colors,
        startangle=90
    )
    ax.set_title(blend_name, fontsize=10, fontweight='bold')

plt.suptitle('Blend Weight Distributions', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 8. Complete Results Comparison

In [ ]:
# Combine all results
all_results = []

# Single models
for r in base_results:
    all_results.append({
        'Method': r['Model'], 
        'Type': 'Single Model', 
        'CV RMSE': r['CV RMSE'], 
        'Std': r['Std']
    })

# Voting
for r in voting_results:
    all_results.append({
        'Method': r['Ensemble'], 
        'Type': 'Voting', 
        'CV RMSE': r['CV RMSE'], 
        'Std': r['Std']
    })

# Stacking
for r in stacking_results:
    all_results.append({
        'Method': r['Ensemble'], 
        'Type': 'Stacking', 
        'CV RMSE': r['CV RMSE'], 
        'Std': r['Std']
    })

# Blending
for r in blend_results:
    all_results.append({
        'Method': r['Ensemble'], 
        'Type': 'Blending', 
        'CV RMSE': r['CV RMSE'], 
        'Std': r['Std']
    })

results_df = pd.DataFrame(all_results).sort_values('CV RMSE')

print("="*70)
print("ALL METHODS COMPARISON (Top 15, sorted by CV RMSE)")
print("="*70)
print(results_df.head(15).to_string(index=False))

In [ ]:
# Comprehensive visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Plot 1: Top 12 methods
ax1 = axes[0]
top_12 = results_df.head(12)
colors = {
    'Single Model': 'steelblue', 
    'Voting': 'coral', 
    'Stacking': 'green', 
    'Blending': 'purple'
}
bar_colors = [colors[t] for t in top_12['Type']]

y_pos = np.arange(len(top_12))
bars = ax1.barh(y_pos, top_12['CV RMSE'], xerr=top_12['Std'], 
                color=bar_colors, edgecolor='black', capsize=3, alpha=0.8)

ax1.set_yticks(y_pos)
ax1.set_yticklabels(top_12['Method'], fontsize=9)
ax1.set_xlabel('CV RMSE (lower is better)', fontsize=11)
ax1.set_title('Top 12 Methods by CV RMSE', fontsize=12, fontweight='bold')
ax1.invert_yaxis()

# Add value labels
for i, val in enumerate(top_12['CV RMSE']):
    ax1.text(val + 0.08, i, f'{val:.3f}', va='center', fontsize=9)

# Legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=colors[k], label=k) for k in colors]
ax1.legend(handles=legend_elements, loc='lower right', fontsize=9)

# Plot 2: Average by method type
ax2 = axes[1]
type_avg = results_df.groupby('Type')['CV RMSE'].agg(['mean', 'std', 'min']).reset_index()
type_avg = type_avg.sort_values('mean')

bars2 = ax2.bar(type_avg['Type'], type_avg['mean'], yerr=type_avg['std'],
                color=[colors[t] for t in type_avg['Type']], 
                edgecolor='black', capsize=5, alpha=0.8)

# Add best in each category
for i, (bar, best) in enumerate(zip(bars2, type_avg['min'])):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.15, 
             f'avg: {bar.get_height():.2f}\nbest: {best:.2f}', 
             ha='center', va='bottom', fontsize=9)

ax2.set_ylabel('Average CV RMSE', fontsize=11)
ax2.set_title('Performance by Method Type', fontsize=12, fontweight='bold')
ax2.set_ylim(10, 12)

plt.tight_layout()
plt.show()

---
## 9. Generate Submission Files

Let's create submission files for the most promising methods.

In [ ]:
# Train final models on full training data
print("Training final models on full data...")

final_models = {}

# Linear models (scaled)
final_models['Ridge'] = Ridge(alpha=1.0)
final_models['Ridge'].fit(X_scaled.values, y.values)

final_models['Lasso'] = Lasso(alpha=0.1)
final_models['Lasso'].fit(X_scaled.values, y.values)

final_models['ElasticNet'] = ElasticNet(alpha=0.1, l1_ratio=0.5)
final_models['ElasticNet'].fit(X_scaled.values, y.values)

# Tree models (raw)
final_models['RandomForest'] = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
final_models['RandomForest'].fit(X.values, y.values)

final_models['GradientBoosting'] = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42)
final_models['GradientBoosting'].fit(X.values, y.values)

# Get test predictions from each model
test_preds = {}
test_preds['Ridge'] = final_models['Ridge'].predict(X_test_scaled.values)
test_preds['Lasso'] = final_models['Lasso'].predict(X_test_scaled.values)
test_preds['ElasticNet'] = final_models['ElasticNet'].predict(X_test_scaled.values)
test_preds['RandomForest'] = final_models['RandomForest'].predict(X_test.values)
test_preds['GradientBoosting'] = final_models['GradientBoosting'].predict(X_test.values)

print("All models trained!")

In [ ]:
# Generate submission files
print("="*60)
print("GENERATING SUBMISSION FILES")
print("="*60)

submissions = {}

# 1. Single best model (ElasticNet)
submissions['elasticnet_single'] = test_preds['ElasticNet']

# 2. Linear blend (Ridge + Lasso + ElasticNet)
submissions['blend_linear'] = (0.33 * test_preds['Ridge'] + 
                                0.33 * test_preds['Lasso'] + 
                                0.34 * test_preds['ElasticNet'])

# 3. Heavy ElasticNet blend
submissions['blend_heavy_en'] = (0.15 * test_preds['Ridge'] + 
                                  0.15 * test_preds['Lasso'] + 
                                  0.5 * test_preds['ElasticNet'] + 
                                  0.1 * test_preds['RandomForest'] + 
                                  0.1 * test_preds['GradientBoosting'])

# 4. ElasticNet + RF (60/40)
submissions['blend_en_rf_60_40'] = (0.6 * test_preds['ElasticNet'] + 
                                     0.4 * test_preds['RandomForest'])

# 5. ElasticNet + RF (70/30)
submissions['blend_en_rf_70_30'] = (0.7 * test_preds['ElasticNet'] + 
                                     0.3 * test_preds['RandomForest'])

# 6. All 5 equal
submissions['blend_all5_equal'] = (test_preds['Ridge'] + 
                                    test_preds['Lasso'] + 
                                    test_preds['ElasticNet'] + 
                                    test_preds['RandomForest'] + 
                                    test_preds['GradientBoosting']) / 5

# Save submission files
for name, preds in submissions.items():
    submission = pd.DataFrame({'Id': test_fe['Id'], 'pop': preds})
    filename = f'./submission_{name}.csv'
    submission.to_csv(filename, index=False)
    print(f"Saved: {filename}")

print("\nAll submissions saved!")

In [ ]:
# Visualize prediction distributions
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

for i, (name, preds) in enumerate(submissions.items()):
    ax = axes[i]
    ax.hist(preds, bins=20, color='steelblue', edgecolor='black', alpha=0.7)
    ax.axvline(preds.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {preds.mean():.1f}')
    ax.set_xlabel('Predicted Popularity')
    ax.set_ylabel('Count')
    ax.set_title(name.replace('_', ' ').title(), fontsize=10, fontweight='bold')
    ax.legend(fontsize=8)

plt.suptitle('Prediction Distributions by Submission', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 10. Summary and Recommendations

In [ ]:
print("="*70)
print("ENSEMBLE METHODS SUMMARY")
print("="*70)

print("\n1. BEST METHODS BY CV RMSE:")
print("-" * 40)
print(results_df.head(5).to_string(index=False))

print("\n2. KEY FINDINGS:")
print("-" * 40)
print("- Single ElasticNet remains competitive with ensembles")
print("- Blending with ElasticNet + small RF contribution works well")
print("- Stacking tends to overfit on this small dataset")
print("- Linear models consistently outperform tree-based models")

print("\n3. RECOMMENDATION FOR KAGGLE:")
print("-" * 40)
print("Try these submissions (in order of priority):")
print("  1. submission_blend_en_rf_60_40.csv (combines linear + non-linear)")
print("  2. submission_blend_heavy_en.csv (diversified blend)")
print("  3. submission_elasticnet_single.csv (best single model)")

print("\n4. WHY CV != KAGGLE SCORE:")
print("-" * 40)
print("- Test set may have different distribution")
print("- Small dataset = high variance in estimates")
print("- Different models may generalize differently")

---
## Key Takeaways

1. **Ensemble methods combine multiple models** to potentially capture different patterns.

2. **Three main approaches:**
   - **Voting:** Simple average (equal weights)
   - **Stacking:** Meta-learner combines predictions
   - **Blending:** Custom weighted average

3. **On this dataset:**
   - Linear models dominate (ElasticNet, Ridge, Lasso)
   - Ensembles don't dramatically improve CV scores
   - But blending linear + tree models may generalize better

4. **Practical advice:**
   - CV performance is a guide, not a guarantee
   - Try multiple submissions to find what works on the test set
   - Diversity in model types can help generalization